# Example for:
Transfer Learning Empirical Experiment from all 10 classes of FashionMNIST into all 10 classes of MNIST 

### Setup and Hyperparams

In [1]:
# Specify which gpu
import os
gpu_id = 5
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

import sys
sys.path.append('/home/arnisaf/mp-tl-study')
from functions.utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)  # if using multi-GPU

In [2]:
# cuts=0 means: end-to-end model if we are reinitializing
cuts = [0,1,2,3,4,5,6]

In [3]:
# Changes Here for the experiments
params = {
      # MODEL ARCHITECTURE PARAMS
      'depth': 6,
      'num_channels': 64,
      'kernel_size': 3,
      'activation_function': nn.ReLU,
      # TRAINING PARAMS
      'device': device,
      'lr_pretrain': 0.001,   
      'lr_fine_tune': 0.001, 
      'num_train': 40,
      'early_stop_patience': 6,
      'batch_size':4096,
      # DATASET PARAMS
      'pre_train_classes': list(range(10)),
      'fine_tune_classes': list(range(10)),
      # EXPERIMENT SETTING PARAMS
      'use_pooling': True,   # CHANGE
      'pooling_every_n_layers': 2, # add pooling after every n layers specified here. For only one pooling after all the CNN layers, this equals params['depth']
      # default value for pooling_every_n_layers is 1 (after each cnn layer)
      'pooling_stride': 2,
      'freeze': True,         # CHANGE: freeze the conv layers before the cut
      'reinit': True         # CHANGE: reinit the conv lyers only after the cut
      }

In [4]:
root_dir = './data'  # Specify your data directory here
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataloader_wrapped = TransferLearningWrapper(params, datasets.FashionMNIST, datasets.MNIST, root_dir, transform=transform)

In [5]:
dataloader_wrapped.pretrain_train_loader.dataset.__len__()

54000

In [6]:
train_loader_reduced = reduce_dataset(dataloader_wrapped.finetune_train_loader, 0.001, seed = 0)
train_loader_reduced.dataset.__len__()

50

## Pretraining

In [7]:
#Create DNN model
pretrained_model = CustomCNN(params, dataloader_wrapped.output_dim)
pretrained_model.to(device)

CustomCNN(
  (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act0): ReLU()
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): ReLU()
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act5): ReLU()
  (pool5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=576, out_features=10, bias=True)
)

In [8]:
# Train and evaluate - Skip if loading saved model!
trainer = Trainer(pretrained_model, dataloader_wrapped, params["lr_pretrain"], params)
train_acc, test_acc, effective_epochs, checkpoints = trainer.train()

print(f"Final Training Accuracy: {train_acc:.4f}")
print(f"Final Test Accuracy: {test_acc:.4f}")

Epoch: 0 	Training Accuracy: 50.89%
Validation Accuracy: 50.18%
Epoch: 1 	Training Accuracy: 66.44%
Validation Accuracy: 65.82%
Epoch: 2 	Training Accuracy: 73.80%
Validation Accuracy: 73.12%
Epoch: 3 	Training Accuracy: 74.74%
Validation Accuracy: 74.50%
Epoch: 4 	Training Accuracy: 74.98%
Validation Accuracy: 74.92%
Epoch: 5 	Training Accuracy: 76.63%
Validation Accuracy: 76.57%
Epoch: 6 	Training Accuracy: 77.45%
Validation Accuracy: 77.40%
Epoch: 7 	Training Accuracy: 78.08%
Validation Accuracy: 77.83%
Epoch: 8 	Training Accuracy: 78.84%
Validation Accuracy: 78.67%
Epoch: 9 	Training Accuracy: 79.90%
Validation Accuracy: 79.78%
Epoch: 10 	Training Accuracy: 81.09%
Validation Accuracy: 80.85%
Epoch: 11 	Training Accuracy: 82.20%
Validation Accuracy: 81.83%
Epoch: 12 	Training Accuracy: 82.57%
Validation Accuracy: 82.25%
Epoch: 13 	Training Accuracy: 82.78%
Validation Accuracy: 82.53%
Epoch: 14 	Training Accuracy: 79.65%
Validation Accuracy: 79.30%
Epoch: 15 	Training Accuracy: 83.08

In [8]:
pretrained_model.load_state_dict(torch.load('pretrained_models/pretrained_model_only_fashion_new.pth'))
pretrained_model.to(device)

CustomCNN(
  (conv0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act0): ReLU()
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): ReLU()
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act5): ReLU()
  (pool5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=576, out_features=10, bias=True)
)

In [9]:
eval(pretrained_model, device, dataloader_wrapped.val_loader, debug=True, classification_report_flag=True, is_cnn=True)


Average loss: 0.3068, Accuracy: 5350.0/6000 (89%)

              precision    recall  f1-score   support

     Class 0       0.84      0.81      0.83       658
     Class 1       0.99      0.99      0.99       611
     Class 2       0.83      0.88      0.86       612
     Class 3       0.84      0.91      0.88       562
     Class 4       0.86      0.81      0.83       607
     Class 5       0.96      0.97      0.96       613
     Class 6       0.74      0.70      0.72       600
     Class 7       0.94      0.95      0.94       537
     Class 8       0.96      0.96      0.96       598
     Class 9       0.97      0.95      0.96       602

    accuracy                           0.89      6000
   macro avg       0.89      0.89      0.89      6000
weighted avg       0.89      0.89      0.89      6000



0.8916666666666667

In [16]:
# save model for later use
foldername = "pretrained_models/pretrained_Fashion_all_classes"
os.mkdir(foldername)
torch.save(pretrained_model.state_dict(), os.path.join(foldername, 'pretrained_model.pth'))

params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
#save params as well
with open(os.path.join(foldername, 'params.json'), 'w') as fp:
    json.dump(params_tmp, fp)

## Fine-tuning Experiments

### Baselines (End-to-end models trained on subsets of fine-tuning dataset)
We also reuse the baselines a lot! so skip if we already have the jsons

In [11]:
baselines_results = []
percentages = [0.001, 0.01, 0.1, 0.5, 1.0]

In [12]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:      
    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
    
    for repeat in range(repeats):
        # Print or log the sampled values for transparency
        print(f"\nSampled Percentage: {sampled_percentage}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

        # Reduce the dataset
        train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed = repeat)
        torch.manual_seed(repeat)
        #train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed = repeat)
        dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)

        # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
        params_tmp = copy.deepcopy(params)
        params_tmp["reinit"] = True
        model_new = cut_custom_cnn_model(pretrained_model, cut_point=0, params=params_tmp, output_dim=dataloader_wrapped.output_dim)
        model_new.to(device)

        # Train and evaluate
        trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
        train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
        print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

        # Store the results
        baselines_results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":-1, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc}) # -1 for the cut point means it's baseline


Sampled Percentage: 0.001, Lr: 0.001, Repeat: 0


In [ ]:
print(baselines_results)

In [21]:
# save baseline results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + baselines_results

with open(f'results_jsons/baselines_freeze_{params["freeze"]}_pool_{params["use_pooling"]}_truncate_{params["truncate"]}.json', 'w') as f:
    json.dump(results, f)

In [17]:
# save baseline results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + baselines_results

with open(f'results/cifarstyle_baselines_fashiontomnist.json', 'w') as f:
    json.dump(results, f)

### Fine-tuning

In [10]:
results = []
#percentages = [0.001, 0.01, 0.1, 0.5, 1.0]
percentages = [1.0]

In [11]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:

    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
        
    for sampled_cut_point in cuts:

        for repeat in range(repeats):
            # Add the combination to the tested set
            # tested_combinations.add((sampled_percentage, sampled_cut_point))

            # Print or log the sampled values for transparency
            print(f"\nSampled Percentage: {sampled_percentage}, Sampled Cut Point: {sampled_cut_point}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

            # Reduce the dataset
            train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed=repeat)
            dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)
            torch.manual_seed(repeat) # because in the cut function we reinitialize some layers too (at least the dense layers)
            
            # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
            model_new = cut_custom_cnn_model(pretrained_model, cut_point=sampled_cut_point, params=params, output_dim=dataloader_wrapped.output_dim)
            model_new.to(device)
            
            # Train and evaluate
            trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
            train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
            print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

            # Store the results
            results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":sampled_cut_point, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc})


Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 0
Early stopping invoked.
Training Accuracy: 0.9875, Test Accuracy: 0.9873

Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 1
Training Accuracy: 0.9912, Test Accuracy: 0.9904

Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 2
Training Accuracy: 0.9917, Test Accuracy: 0.9910

Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 3
Early stopping invoked.
Training Accuracy: 0.9885, Test Accuracy: 0.9882

Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 4
Early stopping invoked.
Training Accuracy: 0.9880, Test Accuracy: 0.9877

Sampled Percentage: 1.0, Sampled Cut Point: 1, Lr: 0.001, Repeat: 0
Training Accuracy: 0.9874, Test Accuracy: 0.9868

Sampled Percentage: 1.0, Sampled Cut Point: 1, Lr: 0.001, Repeat: 1
Training Accuracy: 0.9912, Test Accuracy: 0.9906

Sampled Percentage: 1.0, Sampled Cut Point: 1, Lr: 0.001, Repeat: 2
Training Accuracy: 0.9910, Test A

In [12]:
print(results)

[{'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.987548579406105, 'test_acc': 0.9873}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9911896766403804, 'test_acc': 0.9904333333333334}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9916613213598461, 'test_acc': 0.99105}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.9884918688450364, 'test_acc': 0.9882}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.9880390899143493, 'test_acc': 0.9876666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.9873599215183186, 'test_acc': 0.9868333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9912274082179376, 'test_acc': 0.99055}, {'lr': 0.001, 'sampled_percentage': 1.

In [13]:
print(results)

[{'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.987548579406105, 'test_acc': 0.9873}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9911896766403804, 'test_acc': 0.9904333333333334}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9916613213598461, 'test_acc': 0.99105}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.9884918688450364, 'test_acc': 0.9882}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.9880390899143493, 'test_acc': 0.9876666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.9873599215183186, 'test_acc': 0.9868333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9912274082179376, 'test_acc': 0.99055}, {'lr': 0.001, 'sampled_percentage': 1.

In [1]:
baselines_results = [{'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.88, 'test_acc': 0.5850166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.86, 'test_acc': 0.6236166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.66675}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.98, 'test_acc': 0.6223666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.60335}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 5, 'train_acc': 0.98, 'test_acc': 0.6040166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 6, 'train_acc': 0.66, 'test_acc': 0.5054333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 7, 'train_acc': 0.9, 'test_acc': 0.6562666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 8, 'train_acc': 1.0, 'test_acc': 0.6923666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.6216333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 10, 'train_acc': 0.96, 'test_acc': 0.6587166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 11, 'train_acc': 0.2, 'test_acc': 0.22381666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 12, 'train_acc': 0.92, 'test_acc': 0.6432833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 13, 'train_acc': 0.96, 'test_acc': 0.5967166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 14, 'train_acc': 0.82, 'test_acc': 0.6395666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.68205}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 16, 'train_acc': 0.92, 'test_acc': 0.6336}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 17, 'train_acc': 0.94, 'test_acc': 0.6453666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 18, 'train_acc': 0.9, 'test_acc': 0.6827166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 19, 'train_acc': 0.92, 'test_acc': 0.6413833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.6359166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.5779833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 22, 'train_acc': 0.68, 'test_acc': 0.5370333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 23, 'train_acc': 0.76, 'test_acc': 0.5656666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.6277833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.8685185185185185, 'test_acc': 0.8308}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.5740740740740741, 'test_acc': 0.5586}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.8796296296296297, 'test_acc': 0.84995}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.8685185185185185, 'test_acc': 0.82825}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.8333333333333334, 'test_acc': 0.8067666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 5, 'train_acc': 0.524074074074074, 'test_acc': 0.5236333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 6, 'train_acc': 0.5388888888888889, 'test_acc': 0.5523}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 7, 'train_acc': 0.8648148148148148, 'test_acc': 0.8136166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 8, 'train_acc': 0.5537037037037037, 'test_acc': 0.5436666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 9, 'train_acc': 0.8555555555555555, 'test_acc': 0.8042666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 10, 'train_acc': 0.5907407407407408, 'test_acc': 0.55715}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 11, 'train_acc': 0.8777777777777778, 'test_acc': 0.8239833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 12, 'train_acc': 0.8055555555555556, 'test_acc': 0.7700333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 13, 'train_acc': 0.8555555555555555, 'test_acc': 0.81965}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 14, 'train_acc': 0.8611111111111112, 'test_acc': 0.8345166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 15, 'train_acc': 0.8666666666666667, 'test_acc': 0.8283333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 16, 'train_acc': 0.6277777777777778, 'test_acc': 0.5863833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 17, 'train_acc': 0.8685185185185185, 'test_acc': 0.8080833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 18, 'train_acc': 0.6481481481481481, 'test_acc': 0.6348833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 19, 'train_acc': 0.5611111111111111, 'test_acc': 0.58565}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 20, 'train_acc': 0.8314814814814815, 'test_acc': 0.8137333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 21, 'train_acc': 0.8555555555555555, 'test_acc': 0.7964333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 22, 'train_acc': 0.8888888888888888, 'test_acc': 0.8288666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 23, 'train_acc': 0.5648148148148148, 'test_acc': 0.5655333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 24, 'train_acc': 0.8833333333333333, 'test_acc': 0.81965}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.9375925925925926, 'test_acc': 0.9319}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.9372222222222222, 'test_acc': 0.9288666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.9383333333333334, 'test_acc': 0.9273166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.9348148148148148, 'test_acc': 0.9292666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.932037037037037, 'test_acc': 0.9251833333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 5, 'train_acc': 0.9359259259259259, 'test_acc': 0.9249}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 6, 'train_acc': 0.9248148148148149, 'test_acc': 0.9219}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 7, 'train_acc': 0.9383333333333334, 'test_acc': 0.93085}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 8, 'train_acc': 0.9218518518518518, 'test_acc': 0.9145666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 9, 'train_acc': 0.9424074074074074, 'test_acc': 0.9306333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 10, 'train_acc': 0.9312962962962963, 'test_acc': 0.9199666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 11, 'train_acc': 0.9377777777777778, 'test_acc': 0.93455}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 12, 'train_acc': 0.9416666666666667, 'test_acc': 0.9314833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 13, 'train_acc': 0.9342592592592592, 'test_acc': 0.9264833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 14, 'train_acc': 0.9283333333333333, 'test_acc': 0.9225166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 15, 'train_acc': 0.9281481481481482, 'test_acc': 0.9223833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 16, 'train_acc': 0.9283333333333333, 'test_acc': 0.9205166666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 17, 'train_acc': 0.9338888888888889, 'test_acc': 0.9209666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 18, 'train_acc': 0.9448148148148148, 'test_acc': 0.9363}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 19, 'train_acc': 0.932962962962963, 'test_acc': 0.9227333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.9848888888888889, 'test_acc': 0.9818666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.9710740740740741, 'test_acc': 0.9700333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.9859629629629629, 'test_acc': 0.9825333333333334}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.9825925925925926, 'test_acc': 0.97855}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.9858148148148148, 'test_acc': 0.981}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.987548579406105, 'test_acc': 0.9873}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.9911896766403804, 'test_acc': 0.9904333333333334}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.9916613213598461, 'test_acc': 0.99105}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.9884918688450364, 'test_acc': 0.9882}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.9880390899143493, 'test_acc': 0.9876666666666667}]
finetune1 = [{'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.88, 'test_acc': 0.5850166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.86, 'test_acc': 0.6236166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.66675}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.98, 'test_acc': 0.6223666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.60335}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 5, 'train_acc': 0.98, 'test_acc': 0.6040166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 6, 'train_acc': 0.66, 'test_acc': 0.5054333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 7, 'train_acc': 0.9, 'test_acc': 0.6562666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 8, 'train_acc': 1.0, 'test_acc': 0.6923666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.6216333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 10, 'train_acc': 0.96, 'test_acc': 0.6587166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 11, 'train_acc': 0.2, 'test_acc': 0.22381666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 12, 'train_acc': 0.92, 'test_acc': 0.6432833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 13, 'train_acc': 0.96, 'test_acc': 0.5967166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 14, 'train_acc': 0.82, 'test_acc': 0.6395666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.68205}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 16, 'train_acc': 0.92, 'test_acc': 0.6336}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 17, 'train_acc': 0.94, 'test_acc': 0.6453666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 18, 'train_acc': 0.9, 'test_acc': 0.6827166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 19, 'train_acc': 0.92, 'test_acc': 0.6413833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.6359166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.5779833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 22, 'train_acc': 0.68, 'test_acc': 0.5370333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 23, 'train_acc': 0.76, 'test_acc': 0.5656666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.6277833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.9, 'test_acc': 0.6352166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.88, 'test_acc': 0.60785}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.94, 'test_acc': 0.66215}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.96, 'test_acc': 0.652}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.7, 'test_acc': 0.49061666666666665}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 5, 'train_acc': 0.98, 'test_acc': 0.6312666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 6, 'train_acc': 0.58, 'test_acc': 0.5043166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 7, 'train_acc': 0.8, 'test_acc': 0.6244833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 8, 'train_acc': 0.68, 'test_acc': 0.5128166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 9, 'train_acc': 0.98, 'test_acc': 0.5606833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 10, 'train_acc': 1.0, 'test_acc': 0.6441333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.5728}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.65665}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 13, 'train_acc': 0.9, 'test_acc': 0.5930166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 14, 'train_acc': 0.7, 'test_acc': 0.5745}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 15, 'train_acc': 0.96, 'test_acc': 0.6850833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 16, 'train_acc': 0.74, 'test_acc': 0.52395}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 17, 'train_acc': 0.7, 'test_acc': 0.4996333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 18, 'train_acc': 0.56, 'test_acc': 0.5203833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 19, 'train_acc': 0.96, 'test_acc': 0.6282833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.6422833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 21, 'train_acc': 0.96, 'test_acc': 0.5943666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.6767333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 23, 'train_acc': 0.86, 'test_acc': 0.5593833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 24, 'train_acc': 0.98, 'test_acc': 0.6475333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.74, 'test_acc': 0.5704}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.68, 'test_acc': 0.4941}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.698}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.9, 'test_acc': 0.6193833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.96, 'test_acc': 0.6235333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.6331166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 6, 'train_acc': 0.98, 'test_acc': 0.6523833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 7, 'train_acc': 0.64, 'test_acc': 0.4850333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 8, 'train_acc': 0.72, 'test_acc': 0.5504}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 9, 'train_acc': 0.64, 'test_acc': 0.45498333333333335}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 10, 'train_acc': 0.66, 'test_acc': 0.53845}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.7145333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.6770166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.6105833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 14, 'train_acc': 0.98, 'test_acc': 0.6815333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.69395}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 16, 'train_acc': 0.72, 'test_acc': 0.5710833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.6755166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 18, 'train_acc': 0.62, 'test_acc': 0.5369666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 19, 'train_acc': 0.86, 'test_acc': 0.6793}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 20, 'train_acc': 0.94, 'test_acc': 0.6286333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 21, 'train_acc': 0.94, 'test_acc': 0.5541666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.6171}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 23, 'train_acc': 0.72, 'test_acc': 0.54955}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.6445833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.7027833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.8, 'test_acc': 0.6421}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.71515}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.86, 'test_acc': 0.66175}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.7, 'test_acc': 0.5460833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 5, 'train_acc': 0.84, 'test_acc': 0.5932166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.7351666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 7, 'train_acc': 0.68, 'test_acc': 0.63385}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 8, 'train_acc': 1.0, 'test_acc': 0.7592666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.6371333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 10, 'train_acc': 1.0, 'test_acc': 0.7637666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 11, 'train_acc': 0.64, 'test_acc': 0.463}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.7473666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 13, 'train_acc': 0.8, 'test_acc': 0.6120333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 14, 'train_acc': 0.72, 'test_acc': 0.564}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 15, 'train_acc': 0.74, 'test_acc': 0.5515166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 16, 'train_acc': 0.7, 'test_acc': 0.5821666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 17, 'train_acc': 0.88, 'test_acc': 0.6426}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 18, 'train_acc': 0.72, 'test_acc': 0.5789}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 19, 'train_acc': 0.78, 'test_acc': 0.6007833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 20, 'train_acc': 0.72, 'test_acc': 0.5824833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.6983}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 22, 'train_acc': 0.72, 'test_acc': 0.5377833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 23, 'train_acc': 1.0, 'test_acc': 0.6997333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 24, 'train_acc': 0.8, 'test_acc': 0.5689333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.7144833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.96, 'test_acc': 0.67185}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.6784833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.96, 'test_acc': 0.7180833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.6830833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.68335}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.73705}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 7, 'train_acc': 1.0, 'test_acc': 0.7158}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 8, 'train_acc': 0.84, 'test_acc': 0.66165}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 9, 'train_acc': 0.96, 'test_acc': 0.65045}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 10, 'train_acc': 0.88, 'test_acc': 0.7309666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.6955833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.7171166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.6318166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 14, 'train_acc': 0.84, 'test_acc': 0.64235}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 15, 'train_acc': 0.96, 'test_acc': 0.6863}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.72995}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.73885}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.7307833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 19, 'train_acc': 0.92, 'test_acc': 0.6441833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 20, 'train_acc': 0.98, 'test_acc': 0.71605}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.6975}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.7131166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 23, 'train_acc': 1.0, 'test_acc': 0.6777833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 24, 'train_acc': 0.98, 'test_acc': 0.6856333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.7281666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.98, 'test_acc': 0.6959}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.70595}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.7091333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.7185166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.7011666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 6, 'train_acc': 0.98, 'test_acc': 0.7293666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 7, 'train_acc': 1.0, 'test_acc': 0.75605}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 8, 'train_acc': 1.0, 'test_acc': 0.7561166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 9, 'train_acc': 0.98, 'test_acc': 0.6798}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 10, 'train_acc': 0.98, 'test_acc': 0.72535}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.71195}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.7410333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.6966333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.73355}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.7457166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.7270666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.75465}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.7448}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 19, 'train_acc': 1.0, 'test_acc': 0.7255333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 20, 'train_acc': 0.98, 'test_acc': 0.7243166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.7122}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.75115}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 23, 'train_acc': 1.0, 'test_acc': 0.7131666666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.7149833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.94, 'test_acc': 0.6832}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.9, 'test_acc': 0.6294}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.8, 'test_acc': 0.6136833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.6618833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.92, 'test_acc': 0.6057833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 5, 'train_acc': 0.94, 'test_acc': 0.59815}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 6, 'train_acc': 0.92, 'test_acc': 0.5837833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 7, 'train_acc': 0.62, 'test_acc': 0.5062333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 8, 'train_acc': 0.58, 'test_acc': 0.45811666666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 9, 'train_acc': 0.84, 'test_acc': 0.5536333333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 10, 'train_acc': 0.96, 'test_acc': 0.6958833333333333}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 11, 'train_acc': 0.92, 'test_acc': 0.5852833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 12, 'train_acc': 0.98, 'test_acc': 0.6242166666666666}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 13, 'train_acc': 0.98, 'test_acc': 0.5953833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 14, 'train_acc': 0.94, 'test_acc': 0.6518}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 15, 'train_acc': 0.96, 'test_acc': 0.6820333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 16, 'train_acc': 0.5, 'test_acc': 0.43633333333333335}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 17, 'train_acc': 0.98, 'test_acc': 0.6792166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 18, 'train_acc': 0.9, 'test_acc': 0.6941833333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 19, 'train_acc': 0.7, 'test_acc': 0.46235}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 20, 'train_acc': 0.96, 'test_acc': 0.6528333333333334}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 21, 'train_acc': 0.96, 'test_acc': 0.64825}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 22, 'train_acc': 0.82, 'test_acc': 0.6157166666666667}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 23, 'train_acc': 0.98, 'test_acc': 0.62805}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 24, 'train_acc': 0.96, 'test_acc': 0.6534}]
finetune2 = [{'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.8685185185185185, 'test_acc': 0.8308}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.5740740740740741, 'test_acc': 0.5586}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.8796296296296297, 'test_acc': 0.84995}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.8685185185185185, 'test_acc': 0.82825}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.8333333333333334, 'test_acc': 0.8067666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 5, 'train_acc': 0.524074074074074, 'test_acc': 0.5236333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 6, 'train_acc': 0.5388888888888889, 'test_acc': 0.5523}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 7, 'train_acc': 0.8648148148148148, 'test_acc': 0.8136166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 8, 'train_acc': 0.5537037037037037, 'test_acc': 0.5436666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 9, 'train_acc': 0.8555555555555555, 'test_acc': 0.8042666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 10, 'train_acc': 0.5907407407407408, 'test_acc': 0.55715}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 11, 'train_acc': 0.8777777777777778, 'test_acc': 0.8239833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 12, 'train_acc': 0.8055555555555556, 'test_acc': 0.7700333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 13, 'train_acc': 0.8555555555555555, 'test_acc': 0.81965}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 14, 'train_acc': 0.8611111111111112, 'test_acc': 0.8345166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 15, 'train_acc': 0.8666666666666667, 'test_acc': 0.8283333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 16, 'train_acc': 0.6277777777777778, 'test_acc': 0.5863833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 17, 'train_acc': 0.8685185185185185, 'test_acc': 0.8080833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 18, 'train_acc': 0.6481481481481481, 'test_acc': 0.6348833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 19, 'train_acc': 0.5611111111111111, 'test_acc': 0.58565}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 20, 'train_acc': 0.8314814814814815, 'test_acc': 0.8137333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 21, 'train_acc': 0.8555555555555555, 'test_acc': 0.7964333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 22, 'train_acc': 0.8888888888888888, 'test_acc': 0.8288666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 23, 'train_acc': 0.5648148148148148, 'test_acc': 0.5655333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 24, 'train_acc': 0.8833333333333333, 'test_acc': 0.81965}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.8888888888888888, 'test_acc': 0.84815}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.7981481481481482, 'test_acc': 0.7681}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.5944444444444444, 'test_acc': 0.6093166666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.8388888888888889, 'test_acc': 0.80175}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.6074074074074074, 'test_acc': 0.59725}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 5, 'train_acc': 0.8962962962962963, 'test_acc': 0.8285833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 6, 'train_acc': 0.8314814814814815, 'test_acc': 0.8077833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 7, 'train_acc': 0.8277777777777777, 'test_acc': 0.7883666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 8, 'train_acc': 0.5574074074074075, 'test_acc': 0.5503666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 9, 'train_acc': 0.8555555555555555, 'test_acc': 0.80025}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 10, 'train_acc': 0.8777777777777778, 'test_acc': 0.7960333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 11, 'train_acc': 0.8740740740740741, 'test_acc': 0.8262333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 12, 'train_acc': 0.5388888888888889, 'test_acc': 0.5447}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 13, 'train_acc': 0.8277777777777777, 'test_acc': 0.8097166666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 14, 'train_acc': 0.6055555555555555, 'test_acc': 0.6198166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 15, 'train_acc': 0.8444444444444444, 'test_acc': 0.8139666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 16, 'train_acc': 0.8648148148148148, 'test_acc': 0.7970666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 17, 'train_acc': 0.9, 'test_acc': 0.84065}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 18, 'train_acc': 0.8462962962962963, 'test_acc': 0.8057833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 19, 'train_acc': 0.6351851851851852, 'test_acc': 0.6352166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 20, 'train_acc': 0.837037037037037, 'test_acc': 0.8162666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 21, 'train_acc': 0.8685185185185185, 'test_acc': 0.80055}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 22, 'train_acc': 0.5833333333333334, 'test_acc': 0.5690333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 23, 'train_acc': 0.6111111111111112, 'test_acc': 0.6023833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 24, 'train_acc': 0.8314814814814815, 'test_acc': 0.7892}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.8870370370370371, 'test_acc': 0.8589333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.8703703703703703, 'test_acc': 0.8248166666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.6481481481481481, 'test_acc': 0.6539666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.9018518518518519, 'test_acc': 0.8323833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.8888888888888888, 'test_acc': 0.8482666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 5, 'train_acc': 0.5962962962962963, 'test_acc': 0.5760333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 6, 'train_acc': 0.6722222222222223, 'test_acc': 0.67125}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 7, 'train_acc': 0.8796296296296297, 'test_acc': 0.8433666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 8, 'train_acc': 0.8592592592592593, 'test_acc': 0.8563666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 9, 'train_acc': 0.6592592592592592, 'test_acc': 0.63705}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 10, 'train_acc': 0.9074074074074074, 'test_acc': 0.8493833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 11, 'train_acc': 0.924074074074074, 'test_acc': 0.8726833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 12, 'train_acc': 0.9, 'test_acc': 0.8647166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 13, 'train_acc': 0.8722222222222222, 'test_acc': 0.8509166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 14, 'train_acc': 0.587037037037037, 'test_acc': 0.59545}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 15, 'train_acc': 0.6166666666666667, 'test_acc': 0.6254666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 16, 'train_acc': 0.9018518518518519, 'test_acc': 0.8497333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 17, 'train_acc': 0.8907407407407407, 'test_acc': 0.8561333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 18, 'train_acc': 0.6148148148148148, 'test_acc': 0.6080166666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 19, 'train_acc': 0.6222222222222222, 'test_acc': 0.61335}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 20, 'train_acc': 0.5333333333333333, 'test_acc': 0.50715}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 21, 'train_acc': 0.5888888888888889, 'test_acc': 0.5813666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 22, 'train_acc': 0.9111111111111111, 'test_acc': 0.8448333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 23, 'train_acc': 0.6259259259259259, 'test_acc': 0.6219}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 24, 'train_acc': 0.8944444444444445, 'test_acc': 0.8135166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.9, 'test_acc': 0.8750833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.6685185185185185, 'test_acc': 0.6536}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.9166666666666666, 'test_acc': 0.88075}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.924074074074074, 'test_acc': 0.8512666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.6907407407407408, 'test_acc': 0.6832166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 5, 'train_acc': 0.9351851851851852, 'test_acc': 0.86665}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 6, 'train_acc': 0.5592592592592592, 'test_acc': 0.5589}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 7, 'train_acc': 0.9037037037037037, 'test_acc': 0.8598666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 8, 'train_acc': 0.8759259259259259, 'test_acc': 0.87785}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 9, 'train_acc': 0.6592592592592592, 'test_acc': 0.63745}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 10, 'train_acc': 0.6703703703703704, 'test_acc': 0.6516666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 11, 'train_acc': 0.6037037037037037, 'test_acc': 0.5691}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 12, 'train_acc': 0.6592592592592592, 'test_acc': 0.66545}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 13, 'train_acc': 0.6777777777777778, 'test_acc': 0.6860666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 14, 'train_acc': 0.9018518518518519, 'test_acc': 0.8878}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 15, 'train_acc': 0.9185185185185185, 'test_acc': 0.8887166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 16, 'train_acc': 0.6259259259259259, 'test_acc': 0.6033833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 17, 'train_acc': 0.6685185185185185, 'test_acc': 0.6594166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 18, 'train_acc': 0.9222222222222223, 'test_acc': 0.8722}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 19, 'train_acc': 0.6407407407407407, 'test_acc': 0.63605}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 20, 'train_acc': 0.9018518518518519, 'test_acc': 0.8765833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 21, 'train_acc': 0.6185185185185185, 'test_acc': 0.58325}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 22, 'train_acc': 0.5574074074074075, 'test_acc': 0.5726833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 23, 'train_acc': 0.6370370370370371, 'test_acc': 0.6258333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 24, 'train_acc': 0.7555555555555555, 'test_acc': 0.70975}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.6981481481481482, 'test_acc': 0.6872666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.9018518518518519, 'test_acc': 0.8634666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.8629629629629629, 'test_acc': 0.8497666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.924074074074074, 'test_acc': 0.8503666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.8907407407407407, 'test_acc': 0.8329666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 5, 'train_acc': 0.7351851851851852, 'test_acc': 0.7266666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 6, 'train_acc': 0.8962962962962963, 'test_acc': 0.8444666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 7, 'train_acc': 0.8833333333333333, 'test_acc': 0.8510833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 8, 'train_acc': 0.8611111111111112, 'test_acc': 0.85125}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 9, 'train_acc': 0.8648148148148148, 'test_acc': 0.8348}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 10, 'train_acc': 0.8907407407407407, 'test_acc': 0.8556666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 11, 'train_acc': 0.8648148148148148, 'test_acc': 0.8372333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 12, 'train_acc': 0.8425925925925926, 'test_acc': 0.8124333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 13, 'train_acc': 0.8925925925925926, 'test_acc': 0.8609666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 14, 'train_acc': 0.8962962962962963, 'test_acc': 0.86695}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 15, 'train_acc': 0.9018518518518519, 'test_acc': 0.86355}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 16, 'train_acc': 0.6259259259259259, 'test_acc': 0.6119166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 17, 'train_acc': 0.8907407407407407, 'test_acc': 0.8375333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 18, 'train_acc': 0.7481481481481481, 'test_acc': 0.7089333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 19, 'train_acc': 0.8907407407407407, 'test_acc': 0.8356666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 20, 'train_acc': 0.8166666666666667, 'test_acc': 0.8031}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 21, 'train_acc': 0.8666666666666667, 'test_acc': 0.83525}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 22, 'train_acc': 0.725925925925926, 'test_acc': 0.7117666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 23, 'train_acc': 0.8555555555555555, 'test_acc': 0.8297666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 24, 'train_acc': 0.912962962962963, 'test_acc': 0.8349666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.912962962962963, 'test_acc': 0.853}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.9037037037037037, 'test_acc': 0.8597666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.8666666666666667, 'test_acc': 0.8448666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.9148148148148149, 'test_acc': 0.83995}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.912962962962963, 'test_acc': 0.8599833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 5, 'train_acc': 0.912962962962963, 'test_acc': 0.8595333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 6, 'train_acc': 0.9074074074074074, 'test_acc': 0.8547}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 7, 'train_acc': 0.9259259259259259, 'test_acc': 0.8695166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 8, 'train_acc': 0.8592592592592593, 'test_acc': 0.8401333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 9, 'train_acc': 0.9055555555555556, 'test_acc': 0.84775}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 10, 'train_acc': 0.8796296296296297, 'test_acc': 0.84805}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 11, 'train_acc': 0.8888888888888888, 'test_acc': 0.8544833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 12, 'train_acc': 0.8814814814814815, 'test_acc': 0.84905}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 13, 'train_acc': 0.8851851851851852, 'test_acc': 0.8644833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 14, 'train_acc': 0.924074074074074, 'test_acc': 0.8667166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 15, 'train_acc': 0.8907407407407407, 'test_acc': 0.8709}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 16, 'train_acc': 0.9055555555555556, 'test_acc': 0.8697833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 17, 'train_acc': 0.924074074074074, 'test_acc': 0.8683666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 18, 'train_acc': 0.9018518518518519, 'test_acc': 0.8648166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 19, 'train_acc': 0.9277777777777778, 'test_acc': 0.8578833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 20, 'train_acc': 0.8814814814814815, 'test_acc': 0.85205}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 21, 'train_acc': 0.8888888888888888, 'test_acc': 0.8335}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 22, 'train_acc': 0.9092592592592592, 'test_acc': 0.8560333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 23, 'train_acc': 0.9111111111111111, 'test_acc': 0.8687166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 24, 'train_acc': 0.9185185185185185, 'test_acc': 0.8529833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.7796296296296297, 'test_acc': 0.75}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.762962962962963, 'test_acc': 0.7421333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.7796296296296297, 'test_acc': 0.7767833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.8277777777777777, 'test_acc': 0.7941}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.8481481481481481, 'test_acc': 0.7892166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 5, 'train_acc': 0.7833333333333333, 'test_acc': 0.7394166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 6, 'train_acc': 0.7148148148148148, 'test_acc': 0.6879833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 7, 'train_acc': 0.7592592592592593, 'test_acc': 0.74695}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 8, 'train_acc': 0.7796296296296297, 'test_acc': 0.7818}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 9, 'train_acc': 0.8240740740740741, 'test_acc': 0.7721333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 10, 'train_acc': 0.8222222222222222, 'test_acc': 0.7955333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 11, 'train_acc': 0.7814814814814814, 'test_acc': 0.7833833333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 12, 'train_acc': 0.7592592592592593, 'test_acc': 0.7210333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 13, 'train_acc': 0.7611111111111111, 'test_acc': 0.7641}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 14, 'train_acc': 0.7777777777777778, 'test_acc': 0.7578166666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 15, 'train_acc': 0.7703703703703704, 'test_acc': 0.7799333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 16, 'train_acc': 0.11296296296296296, 'test_acc': 0.11366666666666667}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 17, 'train_acc': 0.8574074074074074, 'test_acc': 0.7856666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 18, 'train_acc': 0.7777777777777778, 'test_acc': 0.7511333333333333}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 19, 'train_acc': 0.8462962962962963, 'test_acc': 0.7606333333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 20, 'train_acc': 0.7925925925925926, 'test_acc': 0.77525}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 21, 'train_acc': 0.8148148148148148, 'test_acc': 0.7592666666666666}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 22, 'train_acc': 0.7962962962962963, 'test_acc': 0.7626833333333334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 23, 'train_acc': 0.8185185185185185, 'test_acc': 0.77895}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 24, 'train_acc': 0.8277777777777777, 'test_acc': 0.77655}]
finetune3 = [{'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.9375925925925926, 'test_acc': 0.9319}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9372222222222222, 'test_acc': 0.9288666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9383333333333334, 'test_acc': 0.9273166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.9348148148148148, 'test_acc': 0.9292666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.932037037037037, 'test_acc': 0.9251833333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 5, 'train_acc': 0.9359259259259259, 'test_acc': 0.9249}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 6, 'train_acc': 0.9248148148148149, 'test_acc': 0.9219}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 7, 'train_acc': 0.9383333333333334, 'test_acc': 0.93085}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 8, 'train_acc': 0.9218518518518518, 'test_acc': 0.9145666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 9, 'train_acc': 0.9424074074074074, 'test_acc': 0.9306333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 10, 'train_acc': 0.9312962962962963, 'test_acc': 0.9199666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 11, 'train_acc': 0.9377777777777778, 'test_acc': 0.93455}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 12, 'train_acc': 0.9416666666666667, 'test_acc': 0.9314833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 13, 'train_acc': 0.9342592592592592, 'test_acc': 0.9264833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 14, 'train_acc': 0.9283333333333333, 'test_acc': 0.9225166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 15, 'train_acc': 0.9281481481481482, 'test_acc': 0.9223833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 16, 'train_acc': 0.9283333333333333, 'test_acc': 0.9205166666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 17, 'train_acc': 0.9338888888888889, 'test_acc': 0.9209666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 18, 'train_acc': 0.9448148148148148, 'test_acc': 0.9363}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 19, 'train_acc': 0.932962962962963, 'test_acc': 0.9227333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.9524074074074074, 'test_acc': 0.94245}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9318518518518518, 'test_acc': 0.9284833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.8974074074074074, 'test_acc': 0.8889333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.917962962962963, 'test_acc': 0.9093333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.914074074074074, 'test_acc': 0.90565}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 5, 'train_acc': 0.9242592592592592, 'test_acc': 0.9127333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 6, 'train_acc': 0.9194444444444444, 'test_acc': 0.92175}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 7, 'train_acc': 0.9187037037037037, 'test_acc': 0.9147166666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 8, 'train_acc': 0.9196296296296296, 'test_acc': 0.9153833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 9, 'train_acc': 0.932962962962963, 'test_acc': 0.9227166666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 10, 'train_acc': 0.9168518518518518, 'test_acc': 0.9051833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 11, 'train_acc': 0.9437037037037037, 'test_acc': 0.93635}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 12, 'train_acc': 0.9407407407407408, 'test_acc': 0.9302333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 13, 'train_acc': 0.922037037037037, 'test_acc': 0.9170666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 14, 'train_acc': 0.9392592592592592, 'test_acc': 0.9334333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 15, 'train_acc': 0.9481481481481482, 'test_acc': 0.9394666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 16, 'train_acc': 0.9212962962962963, 'test_acc': 0.9163166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 17, 'train_acc': 0.9381481481481482, 'test_acc': 0.9261833333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 18, 'train_acc': 0.9275925925925926, 'test_acc': 0.9243833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 19, 'train_acc': 0.922962962962963, 'test_acc': 0.9141166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.9437037037037037, 'test_acc': 0.9395}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.937037037037037, 'test_acc': 0.92955}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.9412962962962963, 'test_acc': 0.9343833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.9401851851851852, 'test_acc': 0.9298666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.9381481481481482, 'test_acc': 0.9291333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 5, 'train_acc': 0.9348148148148148, 'test_acc': 0.9270833333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 6, 'train_acc': 0.9412962962962963, 'test_acc': 0.9419333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 7, 'train_acc': 0.9388888888888889, 'test_acc': 0.9312166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 8, 'train_acc': 0.932962962962963, 'test_acc': 0.93475}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 9, 'train_acc': 0.9444444444444444, 'test_acc': 0.9387666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 10, 'train_acc': 0.937037037037037, 'test_acc': 0.9305666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 11, 'train_acc': 0.9481481481481482, 'test_acc': 0.93865}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 12, 'train_acc': 0.9485185185185185, 'test_acc': 0.9383166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 13, 'train_acc': 0.9344444444444444, 'test_acc': 0.93025}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 14, 'train_acc': 0.9372222222222222, 'test_acc': 0.9320333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 15, 'train_acc': 0.9331481481481482, 'test_acc': 0.9299}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 16, 'train_acc': 0.9394444444444444, 'test_acc': 0.9363333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 17, 'train_acc': 0.9342592592592592, 'test_acc': 0.9202}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 18, 'train_acc': 0.924074074074074, 'test_acc': 0.92205}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 19, 'train_acc': 0.9196296296296296, 'test_acc': 0.91665}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.9377777777777778, 'test_acc': 0.9356333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.9312962962962963, 'test_acc': 0.9259333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.9435185185185185, 'test_acc': 0.9324}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.9366666666666666, 'test_acc': 0.9267166666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.9361111111111111, 'test_acc': 0.92895}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 5, 'train_acc': 0.942037037037037, 'test_acc': 0.9353833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 6, 'train_acc': 0.9133333333333333, 'test_acc': 0.9131333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 7, 'train_acc': 0.9338888888888889, 'test_acc': 0.9268}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 8, 'train_acc': 0.9312962962962963, 'test_acc': 0.93015}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 9, 'train_acc': 0.9403703703703704, 'test_acc': 0.9317333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 10, 'train_acc': 0.9338888888888889, 'test_acc': 0.9278833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 11, 'train_acc': 0.9383333333333334, 'test_acc': 0.92805}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 12, 'train_acc': 0.9405555555555556, 'test_acc': 0.9305}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 13, 'train_acc': 0.9287037037037037, 'test_acc': 0.9267333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 14, 'train_acc': 0.9403703703703704, 'test_acc': 0.93555}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 15, 'train_acc': 0.9374074074074074, 'test_acc': 0.9323333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 16, 'train_acc': 0.937962962962963, 'test_acc': 0.9373}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 17, 'train_acc': 0.9316666666666666, 'test_acc': 0.9225833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 18, 'train_acc': 0.9375925925925926, 'test_acc': 0.9346833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 19, 'train_acc': 0.9314814814814815, 'test_acc': 0.9256666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.9131481481481482, 'test_acc': 0.9137166666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.9281481481481482, 'test_acc': 0.9201333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.9070370370370371, 'test_acc': 0.8946}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.9325925925925926, 'test_acc': 0.9217333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.92, 'test_acc': 0.9129166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 5, 'train_acc': 0.9131481481481482, 'test_acc': 0.9104333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 6, 'train_acc': 0.9214814814814815, 'test_acc': 0.91795}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 7, 'train_acc': 0.9185185185185185, 'test_acc': 0.9116666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 8, 'train_acc': 0.9116666666666666, 'test_acc': 0.9138333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 9, 'train_acc': 0.9101851851851852, 'test_acc': 0.9051666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 10, 'train_acc': 0.9272222222222222, 'test_acc': 0.9214666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 11, 'train_acc': 0.9174074074074074, 'test_acc': 0.9111}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 12, 'train_acc': 0.9183333333333333, 'test_acc': 0.9103333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 13, 'train_acc': 0.9261111111111111, 'test_acc': 0.9222833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 14, 'train_acc': 0.9307407407407408, 'test_acc': 0.9219833333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 15, 'train_acc': 0.9264814814814815, 'test_acc': 0.9230666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 16, 'train_acc': 0.932037037037037, 'test_acc': 0.9283833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 17, 'train_acc': 0.9238888888888889, 'test_acc': 0.9117833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 18, 'train_acc': 0.9274074074074075, 'test_acc': 0.9212833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 19, 'train_acc': 0.9153703703703704, 'test_acc': 0.9109333333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.925, 'test_acc': 0.9191666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.9233333333333333, 'test_acc': 0.9176}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.9251851851851852, 'test_acc': 0.9119}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.9164814814814815, 'test_acc': 0.9101166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.937037037037037, 'test_acc': 0.9250666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 5, 'train_acc': 0.9238888888888889, 'test_acc': 0.9178166666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 6, 'train_acc': 0.9327777777777778, 'test_acc': 0.92305}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 7, 'train_acc': 0.9335185185185185, 'test_acc': 0.9222666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 8, 'train_acc': 0.9114814814814814, 'test_acc': 0.9101}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 9, 'train_acc': 0.9253703703703704, 'test_acc': 0.9179833333333334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 10, 'train_acc': 0.9231481481481482, 'test_acc': 0.9152833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 11, 'train_acc': 0.9211111111111111, 'test_acc': 0.9137666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 12, 'train_acc': 0.9174074074074074, 'test_acc': 0.9065666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 13, 'train_acc': 0.9262962962962963, 'test_acc': 0.9230666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 14, 'train_acc': 0.9231481481481482, 'test_acc': 0.9185333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 15, 'train_acc': 0.9281481481481482, 'test_acc': 0.9174833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 16, 'train_acc': 0.932962962962963, 'test_acc': 0.9251166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 17, 'train_acc': 0.9344444444444444, 'test_acc': 0.9234833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 18, 'train_acc': 0.9264814814814815, 'test_acc': 0.9209166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 19, 'train_acc': 0.9259259259259259, 'test_acc': 0.9158}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.8485185185185186, 'test_acc': 0.85025}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.8457407407407408, 'test_acc': 0.8440833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.8596296296296296, 'test_acc': 0.8511333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.875, 'test_acc': 0.8670833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.8737037037037036, 'test_acc': 0.8653333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 5, 'train_acc': 0.8542592592592593, 'test_acc': 0.8484166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 6, 'train_acc': 0.845, 'test_acc': 0.8461}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 7, 'train_acc': 0.8485185185185186, 'test_acc': 0.8446833333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 8, 'train_acc': 0.8546296296296296, 'test_acc': 0.8570333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 9, 'train_acc': 0.8646296296296296, 'test_acc': 0.8633333333333333}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 10, 'train_acc': 0.8731481481481481, 'test_acc': 0.867}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 11, 'train_acc': 0.8664814814814815, 'test_acc': 0.86045}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 12, 'train_acc': 0.8505555555555555, 'test_acc': 0.8427666666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 13, 'train_acc': 0.8533333333333334, 'test_acc': 0.8583666666666666}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 14, 'train_acc': 0.8583333333333333, 'test_acc': 0.8555166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 15, 'train_acc': 0.8535185185185186, 'test_acc': 0.85285}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 16, 'train_acc': 0.8538888888888889, 'test_acc': 0.84825}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 17, 'train_acc': 0.8683333333333333, 'test_acc': 0.8598166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 18, 'train_acc': 0.8492592592592593, 'test_acc': 0.8495166666666667}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 19, 'train_acc': 0.8607407407407407, 'test_acc': 0.85295}]
finetune4 = [{'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.9848888888888889, 'test_acc': 0.9818666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9710740740740741, 'test_acc': 0.9700333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9859629629629629, 'test_acc': 0.9825333333333334}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.9825925925925926, 'test_acc': 0.97855}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.9858148148148148, 'test_acc': 0.981}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.9787777777777777, 'test_acc': 0.9774666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9853703703703703, 'test_acc': 0.9832666666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.9863703703703703, 'test_acc': 0.9837}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.9835925925925926, 'test_acc': 0.9797}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.9864074074074074, 'test_acc': 0.9827333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.9803703703703703, 'test_acc': 0.97785}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.9756296296296296, 'test_acc': 0.9739333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.973037037037037, 'test_acc': 0.9714833333333334}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.97, 'test_acc': 0.9683}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.9782222222222222, 'test_acc': 0.9745833333333334}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.9736296296296296, 'test_acc': 0.9728}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.9760740740740741, 'test_acc': 0.9747666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.9692592592592593, 'test_acc': 0.9683}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.9757037037037037, 'test_acc': 0.9738333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.9717407407407408, 'test_acc': 0.9689166666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.9642222222222222, 'test_acc': 0.9636833333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.9686666666666667, 'test_acc': 0.9675833333333334}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.9646296296296296, 'test_acc': 0.9637333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.9658518518518519, 'test_acc': 0.9636}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.9678148148148148, 'test_acc': 0.9645333333333334}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.9672222222222222, 'test_acc': 0.9655666666666667}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.9645185185185186, 'test_acc': 0.9619}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.9601851851851851, 'test_acc': 0.9588833333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.9612962962962963, 'test_acc': 0.9598666666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.9659629629629629, 'test_acc': 0.9630666666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.9221851851851852, 'test_acc': 0.9228666666666666}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.9201111111111111, 'test_acc': 0.9195333333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.924962962962963, 'test_acc': 0.9238833333333333}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.9285555555555556, 'test_acc': 0.9268}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.9301851851851852, 'test_acc': 0.9291166666666667}]
finetune5 = [{'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.987548579406105, 'test_acc': 0.9873}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9911896766403804, 'test_acc': 0.9904333333333334}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9916613213598461, 'test_acc': 0.99105}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.9884918688450364, 'test_acc': 0.9882}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.9880390899143493, 'test_acc': 0.9876666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.9873599215183186, 'test_acc': 0.9868333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9912274082179376, 'test_acc': 0.99055}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.9910198845413727, 'test_acc': 0.99045}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.9898124740595404, 'test_acc': 0.9893833333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.98968041353809, 'test_acc': 0.9893666666666666}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.9835113006074784, 'test_acc': 0.9833666666666666}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.986831679432517, 'test_acc': 0.9863666666666666}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.9868505452212957, 'test_acc': 0.9866333333333334}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.9850016979209901, 'test_acc': 0.98465}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.9851903558087763, 'test_acc': 0.9848166666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.9832094479870204, 'test_acc': 0.9829}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.9829830585216768, 'test_acc': 0.9828333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.9844734558351884, 'test_acc': 0.9842833333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.9833037769309135, 'test_acc': 0.9831333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.9854544768516772, 'test_acc': 0.9851333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.9759461193072483, 'test_acc': 0.9759333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.9804739086141192, 'test_acc': 0.9801666666666666}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.9801531902048825, 'test_acc': 0.9799666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.9796815454854167, 'test_acc': 0.9794666666666667}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.9811153454325925, 'test_acc': 0.9811}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.9774553824095386, 'test_acc': 0.9771666666666666}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.9760970456174772, 'test_acc': 0.9756833333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.9731728483567897, 'test_acc': 0.9730833333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.9700788589970947, 'test_acc': 0.97005}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.9718145115647285, 'test_acc': 0.97185}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.94317624419877, 'test_acc': 0.9435333333333333}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.9404218390370901, 'test_acc': 0.941}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.9440252046938082, 'test_acc': 0.9444}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.945308078330755, 'test_acc': 0.9457}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.9478172282383126, 'test_acc': 0.9480333333333333}]

In [2]:
results = finetune1 + finetune2 + finetune3 + finetune4 + finetune5

In [3]:
results

[{'lr': 0.001,
  'sampled_percentage': 0.001,
  'sampled_cut_point': 0,
  'repeat': 0,
  'train_acc': 0.88,
  'test_acc': 0.5850166666666666},
 {'lr': 0.001,
  'sampled_percentage': 0.001,
  'sampled_cut_point': 0,
  'repeat': 1,
  'train_acc': 0.86,
  'test_acc': 0.6236166666666667},
 {'lr': 0.001,
  'sampled_percentage': 0.001,
  'sampled_cut_point': 0,
  'repeat': 2,
  'train_acc': 1.0,
  'test_acc': 0.66675},
 {'lr': 0.001,
  'sampled_percentage': 0.001,
  'sampled_cut_point': 0,
  'repeat': 3,
  'train_acc': 0.98,
  'test_acc': 0.6223666666666666},
 {'lr': 0.001,
  'sampled_percentage': 0.001,
  'sampled_cut_point': 0,
  'repeat': 4,
  'train_acc': 1.0,
  'test_acc': 0.60335},
 {'lr': 0.001,
  'sampled_percentage': 0.001,
  'sampled_cut_point': 0,
  'repeat': 5,
  'train_acc': 0.98,
  'test_acc': 0.6040166666666666},
 {'lr': 0.001,
  'sampled_percentage': 0.001,
  'sampled_cut_point': 0,
  'repeat': 6,
  'train_acc': 0.66,
  'test_acc': 0.5054333333333333},
 {'lr': 0.001,
  'sampl

In [27]:
# save fine-tuning results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + results

with open(f'results_jsons/results_freeze_{params["freeze"]}_reinit_{params["reinit"]}_pool_{params["use_pooling"]}_truncate_{params["truncate"]}.json', 'w') as f:
    json.dump(results, f)
results = results[1:]

In [18]:
# save fine-tuning results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + results

with open(f'cifarstyle_results_fashiontomnist.json', 'w') as f:
    json.dump(results, f)
results = results[1:]